In [61]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import sys
sys.version
#Import Libraries
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import random
%matplotlib inline
import tensorflow as tf
import shutil
import tensorflow.contrib.learn as tflearn
import tensorflow.contrib.layers as tflayers
from tensorflow.contrib.learn.python.learn import learn_runner
import tensorflow.contrib.metrics as metrics
import tensorflow.contrib.rnn as rnn
from random import shuffle
#TF Version
tf.__version__

#with warnings.catch_warnings():
#    warnings.filterwarnings("ignore",category=FutureWarning)
#    import h5py

#num_periods_output = 4 #to predict
#num_periods_input=8 #input

ALL_Test_Data=[]
ALL_Test_Prediction=[]
Event_Based_StartIndex=16
Number_of_EventBased=21
Number_of_TimeFeatures=16


No_Of_weeks=1

<h5>Preprocessing data</h5>

In [0]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer

def preprocessing(df_,num_features):
    
    if df_.ID[0]!=7 and df_.ID[0]!=8:
        print(df_.Year.unique())
        df=df_[(df_.Year==2017) | (df_.Year==2018)]
        #.isin(years)
        #print(df.loc[df['Year'].isin([2017,2018])])
        print(df.Year.unique())
        print(df.Capacity.unique())
    else:
        df=df_[(df_.Year==2015) | (df_.Year==2016)]
        print(df.Year.unique())
        print(df.Capacity.unique())
    
    '''df_=df[['ID','Occupancy','Year', 'Month', 'Day', 'Hour','Minute', 'Capacity', 
    'DayOfWeek','IsWeekend', 'temperature', 'dew_point', 'humidity', 'wind_speed', 
    'feels_like', 'Status', 'light_snow','snow_shower', 'fog', 
    'thunder', 'mostly_cloudy','rain', 'heavy_rain', 'mist', 'shallow_fog','light_freezing_rain',
    'partly_cloudy', 'haze', 'light_rain', 'rain_shower', 'snow', 'light_snow_shower']]'''
    
    
    # select features
    df=df[['ID','Occupancy','Year', 'Month', 'Day', 'Hour','Minute', 
    'DayOfWeek', 'temperature', 'dew_point', 'humidity', 'wind_speed', 'feels_like', 'Status','IsWeekend', 'light_snow','snow_shower', 'fog', 
    'thunder', 'mostly_cloudy','rain', 'heavy_rain', 'mist', 'shallow_fog','light_freezing_rain', 'partly_cloudy',
     'haze', 'light_rain', 'rain_shower', 'snow', 'light_snow_shower','Events_Rugby','Events_Football','Events_Other_Sport','Events_Exhibitions']]
    
    
    ################################################encoding########################
    df['Occupancy'] = pd.to_numeric(df['Occupancy'],errors='coerce')
    df['Occupancy'] = df['Occupancy'].abs()
    
    Status=df.pop('Status')
    df.loc[:,'Status_filling']=(Status=='Filling')*1.0
    df.loc[:,'Status_static']=(Status=='Static')*1.0
    df.loc[:,'Status_emptying']=(Status=='Emptying')*1.0
    #DayOfWeek=df.pop('DayOfWeek')
    #df.loc[:,'DayOfWeek_0']=(DayOfWeek==0)*1.0
    #df.loc[:,'DayOfWeek_1']=(DayOfWeek==1)*1.0
    #df.loc[:,'DayOfWeek_2']=(DayOfWeek==2)*1.0
    #df.loc[:,'DayOfWeek_3']=(DayOfWeek==3)*1.0
    #df.loc[:,'DayOfWeek_4']=(DayOfWeek==4)*1.0
    #df.loc[:,'DayOfWeek_5']=(DayOfWeek==5)*1.0
    #df.loc[:,'DayOfWeek_6']=(DayOfWeek==6)*1.0
    
    Number_Of_Features=num_features
    df=df.values
    df = df.astype('float32')
    split=num_periods_output+num_periods_input
    
    
    ##################################SPLIT##############################################
    print('LEN DF BEFORE CUTTING ANYTHING',len(df))
     ########################## SPLITTING FOR TESTING & VALIDATION ##########################
    #test_len=np.floor(len(df)*0.2)
    test_val_len=np.floor(len(df)*0.2)
    #mod=test_len%(num_periods_input+num_periods_output)
    mod=test_val_len%(num_periods_input+num_periods_output)
    #let thelength be divisable by 12
    test_val_len=int(test_val_len-mod)
    Test_Val=df[(len(df)-test_val_len):,:]
    
    ############################ VALIDATION & TESTING ##################################
    valid_len=np.floor(len(Test_Val)*0.5)
    Valid=Test_Val[0:(len(Test_Val)-int(valid_len)),:]
    Test=Test_Val[(len(Test_Val)-int(valid_len)):,:]
    
    ########################### SPLITTING FOR TRAIN ###########################
    
    new_cutted_df=df[:(len(df)-test_val_len),:]
    Start_train_index=int(12*24*7*No_Of_weeks)
    #print('instances',Start_train_index)
    remain=len(new_cutted_df)-Start_train_index
    #print('len(new_cutted_df)-Start_train_index',remain)
    #print('len(new_cutted_df)',len(new_cutted_df))
    Train=new_cutted_df[len(new_cutted_df)-Start_train_index:,:]
    #print('Train',Train[0])
    #print('len(Train)',len(Train))
    #print(Train.shape)
    train_len=len(Train)
    mod=train_len%(num_periods_input+num_periods_output)
    #let thelength be divisable by 12
    train_len=int(train_len-mod)
    Train=Train[0:train_len,:]
    #print('len Train',len(Train))
   
        
    #############################  Normalization on train and validation separatly  #############
     
    ID_Train=Train[:,0]
    Train=np.delete(Train,[0],1)
    #x_batches=x_batches.drop(columns=['ID'], axis=1)
    occ_Train=Train[:,0]
    Train=np.delete(Train,[0],1)
    #x_batches=x_batches.drop(columns=['Occupancy'], axis=1)
    #normalizing data
    Train = Train.astype('float32')
    normalizer = Normalizer().fit(Train)
    Train=normalizer.transform(Train)
    
    ID_Valid=Valid[:,0]
    Valid=np.delete(Valid,[0],1)
    #X_test=X_test.drop(columns=['ID'], axis=1)
    occ_Valid=Valid[:,0]
    Valid=np.delete(Valid,[0],1)
    #X_test=X_test.drop(columns=['Occupancy'], axis=1)
    Valid = Valid.astype('float32')
    Valid=normalizer.transform(Valid)
    #------------------
    ID_Train=np.reshape(ID_Train,(len(ID_Train),1))
    occ_Train=np.reshape(occ_Train,(len(occ_Train),1))
    
    Train=np.append(occ_Train, Train, axis=1)
    Train=np.append(ID_Train, Train, axis=1)
    #------------------
    ID_Valid=np.reshape(ID_Valid,(len(ID_Valid),1))
    occ_Valid=np.reshape(occ_Valid,(len(occ_Valid),1))
    
    Valid=np.append(occ_Valid,Valid, axis=1)
    Valid=np.append(ID_Valid, Valid, axis=1)

    ############################################ TRAIN minibatches ##################################
    
    end=len(Train)
    start=0
    next=0
    x_batches=[]
    y_batches=[]
    
    count=0
    #print('lennnn',len(Train))
    while next+(num_periods_input+num_periods_output)<end:
        next=start+num_periods_input
        x_batches.append(Train[start:next,:])
        y_batches.append(Train[next:next+num_periods_output,1])
        start=start+1
    y_batches=np.asarray(y_batches)
    y_batches = y_batches.reshape(-1, num_periods_output, 1)   
    x_batches=np.asarray(x_batches)
    x_batches = x_batches.reshape(-1, num_periods_input, Number_Of_Features)   
    print('len x_batches ',len(x_batches))
    
    ############################################ VALID minibatches ##################################
    
    end_val=len(Valid)
    start_val=0
    next_val=0
    x_validbatches=[]
    y_validbatches=[]
    
    while next_val+(num_periods_input+num_periods_output)<end_val:
        next_val=start_val+num_periods_input
        x_validbatches.append(Valid[start_val:next_val,:])
        y_validbatches.append(Valid[next_val:next_val+num_periods_output,1])
        start_val=start_val+1
    y_validbatches=np.asarray(y_validbatches)
    y_validbatches = y_validbatches.reshape(-1, num_periods_output, 1)   
    x_validbatches=np.asarray(x_validbatches)
    x_validbatches = x_validbatches.reshape(-1, num_periods_input, Number_Of_Features)   

    ###########################################TEST#####################################
    
    ID_Test=Test[:,0]
    Test=np.delete(Test,[0],1)
    #X_test=X_test.drop(columns=['ID'], axis=1)
    occ_Test=Test[:,0]
    Test=np.delete(Test,[0],1)
    #X_test=X_test.drop(columns=['Occupancy'], axis=1)
    Test = Test.astype('float32')
    Test=normalizer.transform(Test)
    
    #------------------
    ID_Test=np.reshape(ID_Test,(len(ID_Test),1))
    occ_Test=np.reshape(occ_Test,(len(occ_Test),1))
    
    Test=np.append(occ_Test,Test, axis=1)
    Test=np.append(ID_Test, Test, axis=1)
    
    ############################################ TEST minibatches ##################################
    end_test=len(Test)
    start_test=0
    next_test=0
    x_testbatches=[]
    y_testbatches=[]
    
    
    #print('lennnn',len(Train))
    while next_test+(num_periods_input+num_periods_output)<end_test:
        next_test=start_test+num_periods_input
        x_testbatches.append(Test[start_test:next_test,:])
        y_testbatches.append(Test[next_test:next_test+num_periods_output,1])
        start_test=start_test+1
    y_testbatches=np.asarray(y_testbatches)
    y_testbatches = y_testbatches.reshape(-1, num_periods_output, 1)   
    x_testbatches=np.asarray(x_testbatches)
    x_testbatches = x_testbatches.reshape(-1, num_periods_input, Number_Of_Features) 
    print('len Test',len(Test))
    print('len xTestbatches',len(x_testbatches))
    ######################## Sampling##########################################
    
    #x_batches, y_batches, x_validbatches, y_validbatches, x_testbatches, y_testbatches
    
    return x_batches, y_batches,x_validbatches, y_validbatches, x_testbatches, y_testbatches

In [0]:
def load_locationfiles(path,loc_id):
    filename=path + '/BN00'+str(loc_id)+'.csv'
    print(filename)
    data_loc=pd.read_csv(filename)
    #mod=len(data_loc)%(num_periods_input+num_periods_output)
    #let thelength be divisable by 12
    #data_loc=data_loc[:len(data_loc)-mod]
    return data_loc

# Creating Tensorflow Graph + Run Session

##### Training Params:

In [0]:
##### Prediction Params #####
num_periods_output = 12    #to predict
num_periods_input=24       #input
#num_periods = 4           #number of periods per vector we are using to predict one period ahead

##### Graph Params #####
inputs = Number_of_TimeFeatures-1                #number of vectors submitted
hidden = 128               #number of neurons we will recursively work through, can be changed to improve accuracy
hidden_event=128
output = 1                 #number of output vectors
num_layers=1
num_layers_EventBased=1

##### Static Features #####
Number_Of_Static_Features=18
no_sequences=256            #re-iterating factor for static features

##### Optimization Params #####
batchsize=256
l_rate = 0.00004          #small learning rate so we don't overshoot the minimum
keep_prob_static=0.7       #number of epochs using the constant init_learning_rate
keep_prob_event=0.7
lamda=0.3                #regularization param 
keep_probab=0.95            #(1-dropout_rate)
keep_prob_testval=1.0      #(1-dropout_rate) for testing and validation runs
epochs = 130                #number of iterations or training cycles, includes both the FeedFoward and Backpropogation


#where to save logs
#logs_path = '/tmp/tf_logs_3_LSTM_Fully_0410_ConnEnd_Shift_FSTop3/example/'

In [0]:
# model decaying learning rate, instead of fixed one
#def compute_LR(init_epoch,max_epoch,learning_rate_decay,init_learning_rate):
#    LR_to_use = [
#        init_learning_rate * (
#            learning_rate_decay ** max(float(i+1-init_epoch),0.0)
#        ) for i in range(max_epoch)
#    ]
#    print("Middle LR:", LR_to_use[len(LR_to_use) // 2])
#    return LR_to_use

In [0]:
train_mse=[]
validation_mse=[]
test_mse=[]
tf.reset_default_graph()   #We didn't have any previous graph objects running, but this would reset the graphs


#=============================LSTM part===========================================

lstm_graph=tf.Graph()
with lstm_graph.as_default():
    X = tf.placeholder(tf.float32, [None, num_periods_input, inputs])   #create variable objects
    y = tf.placeholder(tf.float32, [None, num_periods_output, output])
    learning_rate=tf.placeholder(tf.float32, None)
    #batchsize = tf.placeholder(tf.int16)
    #dropout = tf.placeholder(tf.float32)
    is_train = tf.placeholder(tf.bool)
    EventBased_X=tf.placeholder(tf.float32, [None,num_periods_input,Number_of_EventBased])
    static_X = tf.placeholder(tf.float32, [None,Number_Of_Static_Features])
    keep_prob = tf.placeholder(tf.float32, None)

    
    #====================================================================================
    #=========================Static Features part=======================================

    Layer1 = tf.layers.dense(static_X, units=100,activation=tf.nn.relu)  
    Static_Output_1 = tf.layers.dense(Layer1,units= 50,activation=tf.nn.relu) 
    Static_Output = tf.nn.dropout(Static_Output_1,keep_prob=keep_prob_static) # dropout 
    #===================================== Combine before LSTM ===========================
    Static_Output=tf.reshape(Static_Output, [-1, num_periods_input,50])

    Combined_output=tf.concat([X,Static_Output],2)
    #========================================= LSTM ===========================================
    with tf.variable_scope('Time'):
        #create our RNN object
        cells = []
        for i in range(num_layers):
            LSTM_cell=tf.contrib.rnn.LSTMCell(num_units=hidden, activation=tf.nn.relu)
            #LSTM_cell=tf.contrib.rnn.DropoutWrapper(LSTM_cell, output_keep_prob=keep_prob)
            cells.append(LSTM_cell )

        cells = tf.contrib.rnn.MultiRNNCell(cells,state_is_tuple=True)
        #x_norm = tf.layers.batch_normalization(X, training=is_train)
        rnn_output, states = tf.nn.dynamic_rnn(cells, Combined_output, dtype=tf.float32)               #choose dynamic over static

        # ======= Fully connected after LSTM part==========
        stacked_rnn_output = tf.reshape(rnn_output, [-1,(hidden*num_periods_input)])# to be able to concat. the static features

        LSTM_Output_1 = tf.layers.dense(stacked_rnn_output, units=100,activation=tf.nn.relu)  
        LSTM_Output = tf.nn.dropout(LSTM_Output_1,keep_prob=keep_prob) # dropout
    #========================LSTM Event Based============================================
    with tf.variable_scope('Event'):
        #====================================================================
        cells_event = []
        for i in range(num_layers_EventBased):
            LSTM_cell_event=tf.contrib.rnn.LSTMCell(num_units=hidden_event, activation=tf.nn.relu)
            #LSTM_cell_event=tf.contrib.rnn.DropoutWrapper(LSTM_cell_event, output_keep_prob=keep_prob)
            cells_event.append(LSTM_cell_event )

        cells_event = tf.contrib.rnn.MultiRNNCell(cells_event,state_is_tuple=True)

        rnn_output_event, states_event = tf.nn.dynamic_rnn(cells_event,EventBased_X, dtype=tf.float32)               #choose dynamic over static

        # ======= Fully connected after Event Based LSTM part===========================================
        #===================================================================================
        stacked_rnn_output_event = tf.reshape(rnn_output_event, [-1,(hidden_event*num_periods_input)])# to be able to concat. the static features

        LSTM_Output_event_1 = tf.layers.dense(stacked_rnn_output_event, units=100,activation=tf.nn.relu)  
        LSTM_Output_event = tf.nn.dropout(LSTM_Output_event_1,keep_prob=keep_prob_event)
        #====================================================================================
    #=========================Combination part===========================================

    Combined_output=tf.concat([LSTM_Output,LSTM_Output_event],1)
    stacked_outputs = tf.layers.dense(Combined_output, units=12) #specify the type of layer (dense)
    outputs = tf.reshape(stacked_outputs, [-1, num_periods_output, output])          #shape of results
    #Regularization part
    tv = tf.trainable_variables()
    regularization_cost = tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv ])
    Error=tf.square(outputs - y)
    #Total_err=Error+regularization_cost
    loss = tf.reduce_mean(tf.square(outputs - y))+ lamda*regularization_cost   #define the cost function which evaluates the quality of our model
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)          #gradient descent method
    gvs = optimizer.compute_gradients(loss)
    capped_gvs = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gvs]
    
    #opt to get gradient info:
    grads=tf.gradients(loss,tv)
    grads=list(zip(grads,tv))
    
    training_op = optimizer.apply_gradients(capped_gvs)
    #training_op = optimizer.minimize(loss)          #train the result of the application of the cost_function                                 
    
    #Before running session, set variables to track in Tensorboard
    #with lstm_graph.as_default():
    saver = tf.train.Saver()
    #Creat Summary to monitor scalar values (loss):
    #tf.summary.scalar("loss", loss)
    
    # Create a summary to monitor prediction tensor
    #tf.summary.histogram("Prediction", predictions)
    
    # Gradient Info
    for var in tv:
        tf.summary.histogram(var.name,var)
    
    # Merge all summaries into a single op
    merged_summary_op = tf.summary.merge_all()

#====================================================================================

def batch_features_labels(features, labels, batch_size):
    """
    Split features and labels into batches
    """
    #batch_size=
    X_returned=[]
    Y_returned=[]
    for start in range(0, len(features), batch_size):
        if (start + batch_size)>len(features):
            break
        else:
            end = start + batch_size
        X_returned.append(features[start:end])
        Y_returned.append(labels[start:end])
    return X_returned, Y_returned




Static_Features=[[1,0,0,0,0,0,0,0,1,3,4,6,7,19,9,17,51.3787,-2.3622],[0,1,0,0,0,0,0,0,0,3,9,11,3,29,10,13,51.3843,-2.3686],
                 [0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,51.4113,-2.3869],[0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,0,51.3902,-2.4059],
                 [0,0,0,0,1,0,0,0,0,0,0,4,2,3,0,0,51.3529,-2.3838],[0,0,0,0,0,1,0,0,2,4,8,6,3,39,13,14,51.3842,-2.3590],
                 [0,0,0,0,0,0,1,0,1,1,3,6,6,17,9,7,51.3782,-2.3589],[0,0,0,0,0,0,0,1,1,2,3,6,6,18,9,7,51.3783,-2.3593]]

#Static_Features=[    [1,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0],
#                     [0,0,1,0,0,0,0,0],[0,0,0,1,0,0,0,0]
#                    ,[0,0,0,0,1,0,0,0],[0,0,0,0,0,1,0,0],
#                     [0,0,0,0,0,0,1,0],[0,0,0,0,0,0,0,1]]



# COMPUTE LEARNING RATE schedule beforehand
#LR_schedule = compute_LR(init_epoch,epochs,learning_rate_decay,initial_LR) 
with tf.Session(graph=lstm_graph) as sess:    
    
    init = tf.global_variables_initializer()           #initialize all the variables
    init.run()
    
    # creating the writer inside the session
    #writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    
    #### LOOP OVER LOCATIONS ####
    data_path=r'/content/drive/My Drive/FINAL_DATA_EVENTS'
    #r'/content/drive/My Drive/FINAL_DATA_EVENTS'
    #r'D:\Users\danie\Desktop\jupyter_notebooks\Project\FINAL_DATA_EVENTS'
    data_All=pd.DataFrame()
    x_batches_Full=[]
    y_batches_Full=[]
    X_Valid_Full=[]
    Y_Valid_Full=[]
    X_Test_Full=[]
    Y_Test_Full=[]
    for loc_id in range(1,9):
        #========
        data=load_locationfiles(data_path,loc_id)
        header=list(data.columns.values)
        data=pd.DataFrame(data,columns=header)
        x_batches, y_batches, X_Valid, Y_Valid,X_Test,Y_Test=preprocessing(data,(Number_of_TimeFeatures+Number_of_EventBased))
        #===============================
        for element1 in (x_batches):
            x_batches_Full.append(element1)
            
        for element2 in (y_batches):
            y_batches_Full.append(element2)
            
        for element3 in (X_Valid):
            X_Valid_Full.append(element3)
            
        for element4 in (Y_Valid):
            Y_Valid_Full.append(element4)
            
        for element5 in (X_Test):
            X_Test_Full.append(element5)
            
        for element6 in (Y_Test):
            Y_Test_Full.append(element6)            
    
    for ep in range(epochs):
        #CALCULATE LR TO USE!!
        #curr_lr=LR_schedule[ep]
        Training_Error=[]
        #even though NOT SHUFFLED YET
        shuffled_batch_features,shuffled_batch_y=batch_features_labels(x_batches_Full,y_batches_Full,batchsize)
        #---------------------shuffle minibatches X and Y together-------------------------------------
        combined = list(zip(shuffled_batch_features, shuffled_batch_y))
        random.shuffle(combined)
        shuffled_batch_features[:], shuffled_batch_y[:] = zip(*combined)
        #===========================================================================================
        print(len(shuffled_batch_features))
        
        for i in range(0,len(shuffled_batch_features)): 
            #print('====================')
            batch_features=shuffled_batch_features[i]
            #print(batch_features)
            batch_y=shuffled_batch_y[i]
            static_train=[]
            Event_Based_Train=[]
            
            #print('Before drop',batch_features[0][0][0])
            for h in range(0,no_sequences):
                Event_window=[]
                for k in range(0,num_periods_input):
                    loc_id=int(batch_features[h][k][0])
                    static_train.append(Static_Features[loc_id-1][:]) # 
                    Event_window.append(batch_features[h][k][Event_Based_StartIndex:])                    
                Event_Based_Train.append(Event_window)
                
            EventBased_EndIndex=Event_Based_StartIndex+Number_of_EventBased
            Event_Based_index=list(range(Event_Based_StartIndex, EventBased_EndIndex))
            #print('THE EVENT BASED:  ',Event_Based_index)
            batch_features=np.delete(batch_features,Event_Based_index, axis=2) 
            batch_features=np.delete(batch_features,[0], axis=2) 
            
            #print('X',len(batch_features))
            #print('Y',len(batch_y[0]))
            _,Train_Batch_error,summary=sess.run([training_op,Error,merged_summary_op],
                                                 feed_dict={
                                                     keep_prob:keep_probab,
                                                     X: batch_features,
                                                     y: batch_y,
                                                     learning_rate: l_rate,
                                                     static_X:static_train,
                                                     EventBased_X:Event_Based_Train})  
            Training_Error.append(Train_Batch_error)
            
        
        #write logs (per epoch)
        #writer.add_summary(summary, ep)
        
        if ep % 3 == 0:
            ################  evaluate training error
            #print('Length of whole:',len(Training_Error),'Length of each: ',len(Training_Error[0][0]))
            Sum_train=np.sum(Training_Error,axis=2)
            Sum_train_1=np.sum(Sum_train,axis=1)
            Sum_train_2=np.sum(Sum_train_1,axis=0)
            '''print('Sum_train shape:',Sum_train.shape)
            print('Sum_train_1 shape:',Sum_train_1.shape)
            print('Sum_train_2 shape:',Sum_train_2.shape)'''
            Mean_train=Sum_train_2/(len(Training_Error)*batchsize*num_periods_output)
            print("epoch:",int(ep+1))
            print("\tRMSE Training:", (Mean_train)**0.5)
            #mse = loss.eval(feed_dict={
            #                              X: batch_features,
            #                             y: batch_y,
            #                            static_X:static_train,
            #                           is_train:True})
            train_mse.append((Mean_train)**0.5)
                
            
            Validation_Error=[]
            unshuffled_valid_batch_features,unshuffled_valid_batch_y=batch_features_labels(X_Valid_Full,Y_Valid_Full,batchsize)
            
            #---------------------shuffle minibatches X and Y together-------------------------------------
            #combined = list(zip(unshuffled_valid_batch_features, unshuffled_valid_batch_y))
            #random.shuffle(combined)
            #shuffled_batch_features[:], shuffled_batch_y[:] = zip(*combined)
            #===========================================================================================
            #print(len(unshuffled_valid_batch_features))
            
            for i in range(0,len(unshuffled_valid_batch_features)):
                batch_features_valid=unshuffled_valid_batch_features[i]
                #print(batch_features)
                batch_y_valid=unshuffled_valid_batch_y[i]
                static_Valid=[]
                Event_Based_Valid=[]
                #print('Before drop',batch_features[0][0][0])
                for h in range(0,no_sequences):
                    Event_window_valid=[]
                    for k in range(0,num_periods_input):
                        loc_id=int(batch_features_valid[h][k][0])
                        static_Valid.append(Static_Features[loc_id-1][:]) # 
                        Event_window_valid.append(batch_features_valid[h][k][Event_Based_StartIndex:])
                    Event_Based_Valid.append(Event_window_valid)
                    
                EventBased_EndIndex=Event_Based_StartIndex+Number_of_EventBased
                Event_Based_index=list(range(Event_Based_StartIndex, EventBased_EndIndex))
                batch_features_valid=np.delete(batch_features_valid,Event_Based_index, axis=2)
                batch_features_valid=np.delete(batch_features_valid,[0], axis=2)
                
                
                y_predict,Valid_error_batch=sess.run([outputs,Error], 
                                                     feed_dict={
                                                     keep_prob:keep_prob_testval,
                                                     X: batch_features_valid, 
                                                     y: batch_y_valid,
                                                     static_X:static_Valid,
                                                     EventBased_X:Event_Based_Valid})
                #print('Valid_error_batch',Valid_error_batch)
                Validation_Error.append(Valid_error_batch)
            
            Sum_valid=np.sum(Validation_Error,axis=2)
            Sum_valid_1=np.sum(Sum_valid,axis=1)
            Sum_valid_2=np.sum(Sum_valid_1,axis=0)
            '''print('Sum_train shape:',Sum_valid.shape)
            print('Sum_train_1 shape:',Sum_valid_1.shape)
            print('Sum_train_2 shape:',Sum_valid_2.shape)'''
            Mean_valid=Sum_valid_2/(len(Validation_Error)*batchsize*num_periods_output)
            print("\tRMSE Validation:", (Mean_valid)**0.5)
            #mse = loss.eval(feed_dict={
            #                              X: batch_features,
            #                             y: batch_y,
            #                            static_X:static_train,
            #                           is_train:True})
            #train_mse.append((Mean_train)**0.5)
            
            validation_mse.append((Mean_valid)**0.5)
              
    # save training session --> TO RESTORE
    saver.save(sess,"checkpoints-conn_beg/har.ckpt")
    print("Run the command line:\n" \
          "--> tensorboard --logdir=/tmp/tf_logs_1_LSTM_Fully_1110_ConnBeg_Shift_FS4/example " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")

INFO:tensorflow:Summary name dense/kernel:0 is illegal; using dense/kernel_0 instead.
INFO:tensorflow:Summary name dense/bias:0 is illegal; using dense/bias_0 instead.
INFO:tensorflow:Summary name dense_1/kernel:0 is illegal; using dense_1/kernel_0 instead.
INFO:tensorflow:Summary name dense_1/bias:0 is illegal; using dense_1/bias_0 instead.
INFO:tensorflow:Summary name Time/rnn/multi_rnn_cell/cell_0/lstm_cell/kernel:0 is illegal; using Time/rnn/multi_rnn_cell/cell_0/lstm_cell/kernel_0 instead.
INFO:tensorflow:Summary name Time/rnn/multi_rnn_cell/cell_0/lstm_cell/bias:0 is illegal; using Time/rnn/multi_rnn_cell/cell_0/lstm_cell/bias_0 instead.
INFO:tensorflow:Summary name Time/dense/kernel:0 is illegal; using Time/dense/kernel_0 instead.
INFO:tensorflow:Summary name Time/dense/bias:0 is illegal; using Time/dense/bias_0 instead.
INFO:tensorflow:Summary name Event/rnn/multi_rnn_cell/cell_0/lstm_cell/kernel:0 is illegal; using Event/rnn/multi_rnn_cell/cell_0/lstm_cell/kernel_0 instead.
IN

In [0]:
print('min training RMSE:',min(train_mse))
print('min validation RMSE:',min(validation_mse))

### Plot Training + Validation

In [0]:
Train_draw=[]
Valid_draw=[]
for i in range(0,len(train_mse)):
    if i%1==0:
        Train_draw.append(train_mse[i])
        Valid_draw.append(validation_mse[i])

plt.title("Training vs. Validation Error", fontsize=14)
#plt.legend(loc="lower right")
plt.plot(Train_draw[1:], markersize=10, label="Training Error",color='red')
plt.plot(Valid_draw[1:], markersize=10, label="Validation Error",color='blue')
plt.legend(loc="upper right")
plt.savefig('Training vs Testing Error.png')
plt.show()

### Testing Part

In [0]:
Test_Error=[]
with tf.Session(graph=lstm_graph) as sess: 
    # Restore
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints-conn_beg'))
    test_batch_features,test_batch_y=batch_features_labels(X_Test_Full,Y_Test_Full,batchsize)
    #---------------------shuffle minibatches X and Y together-------------------------------------
    #combined = list(zip(shuffled_batch_features, shuffled_batch_y))
    #random.shuffle(combined)
    #shuffled_batch_features[:], shuffled_batch_y[:] = zip(*combined)
    #===========================================================================================
    print(len(test_batch_features))
    
    for i in range(0,len(test_batch_features)):
        batch_features_test=test_batch_features[i]
        #print(batch_features)
        batch_y_test=test_batch_y[i]
        static_test=[]
        Event_Based_Test=[]
        #print('Before drop',batch_features[0][0][0])
        for h in range(0,no_sequences):
            Event_window_Test=[]
            for k in range(0,num_periods_input):
                loc_id=int(batch_features_test[h][k][0])
                static_test.append(Static_Features[loc_id-1][:]) # 
                Event_window_Test.append(batch_features_test[h][k][Event_Based_StartIndex:])
            Event_Based_Test.append(Event_window_Test)
            
        EventBased_EndIndex=Event_Based_StartIndex+Number_of_EventBased
        Event_Based_index=list(range(Event_Based_StartIndex, EventBased_EndIndex))
        batch_features_test=np.delete(batch_features_test,Event_Based_index, axis=2)  
        batch_features_test=np.delete(batch_features_test, [0], axis=2) 
        #print('X',len(batch_features[0]))
        #print('Y',len(batch_y[0]))
        Test_Batch_error,y_predict_test,summary=sess.run([Error,outputs,merged_summary_op],
                                             feed_dict={
                                                 keep_prob:keep_prob_testval,
                                                 X: batch_features_test,
                                                 y: batch_y_test,
                                                 learning_rate: l_rate,
                                                 static_X:static_test,
                                                 is_train:False,
                                                 EventBased_X:Event_Based_Test})

        #print('error:',Train_Batch_error[0])
        Test_Error.append(Test_Batch_error)

    Sum_test=np.sum(Test_Error,axis=2)
    Sum_test_1=np.sum(Sum_test,axis=1)
    Sum_test_2=np.sum(Sum_test_1,axis=0)
    print('Sum_test shape:',Sum_test.shape)
    print('Sum_test_1 shape:',Sum_test_1.shape)
    print('Sum_test_2 shape:',Sum_test_2.shape) 
    Mean_test=Sum_test_2/(len(Test_Error)*batchsize*num_periods_output) 
    print("\tRMSE Testing:", (Mean_test)**0.5)